## Recreate our Galactic Models FITS, this time as one FITS file (aka model)

In [1]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import astropy.constants as const
import astropy.units as u
import io 

In [2]:
files = []
for file in os.listdir('../data/galaxySEDs/'):
    if file.endswith('.norm_1p6'):
        files.append(file)

## Create FITS

In [5]:
from astropy.io import fits
from astropy import table
from astropy import units as u
from astropy import constants as const

In [14]:
tab = []
for file in files:
    tab.append(pd.read_csv('../data/galaxySEDs/'+file,header = None, delim_whitespace=True))

In [17]:
print(tab[0][0])

0        9.099999e+01
1        9.400000e+01
2        9.599999e+01
3        9.800000e+01
4        1.000000e+02
             ...     
11000    1.909001e+07
11001    1.918999e+07
11002    1.929000e+07
11003    1.938999e+07
11004    1.948999e+07
Name: 0, Length: 11005, dtype: float64


In [ ]:
tab = np.ndarray([pd.read_csv('../data/galaxySEDs/'+file,header = None, delim_whitespace=True) for file in files])
for i in range(len(files)):
    tab = pd.read_csv('../data/galaxySEDs/'+files[i],header=None,delim_whitespace=True)
modname = files[i].split('.')[0]
a1 = np.ones(1)
primhead = fits.Header([fits.Card('DISTANCE',3E+21,'Distance in cm'),
                    fits.Card('NWAV',len(tab),'Number of wavelengths')])
primhdu = fits.PrimaryHDU(a1,primhead)

col1 = fits.Column(name='MODEL_NAME', format='20A', array=[modname]) #Column objects package the arrays with the typical fits info: name and format
cols = fits.ColDefs([col1]) #ColDefs object packages the column objects
namehdu = fits.BinTableHDU.from_columns(cols,name = 'MODEL_NAMES') #Creates the bintablehdu object to be made to FITS_rec

wavlist = (tab[0]/10**4).values
freqlist = const.c.value / wavlist
speccol1 = fits.Column(name='WAVELENGTH',format = 'D', unit = 'um', array=wavlist)
speccol2 = fits.Column(name='FREQUENCY', format = 'D', unit = 'Hz', array=freqlist)
speccols = fits.ColDefs([speccol1,speccol2])
spechdu = fits.BinTableHDU.from_columns(speccols,name = 'SPECTRAL_INFO')

# Not sure if apertures matter or should be the same as for YSOs? I'm thinking that we have 0 apertures because we only have data
# for one model, not across apertures or 10000 models.
#aplist = np.logspace(2,6,20)
#apcol1 = fits.Column(name='APERTURE', format = 'D', unit = 'AU', array=aplist)
#apcols = fits.ColDefs([apcol1])
#aphdu = fits.BinTableHDU.from_columns(apcols,name = 'APERTURES')

# Giving synthetic unit here: mJy, despite not being in mJy, normalized at 1.6 um, index 6396
valdata = np.array((tab[1]).values)
valdata = valdata / valdata[6396]
valdata = valdata.reshape((1,1,11005))
valhdr = fits.Header([fits.Card('BUNIT','mJy','')])
valhdu = fits.ImageHDU(data=valdata,name='VALUES',header = valhdr)

# Uncertainty in the models?? Don't have any anymore
uncdata = np.zeros(11005)
uncdata = uncdata.reshape((1,1,11005))
unchdr = fits.Header([fits.Card('BUNIT','mJy','')])
unchdu = fits.ImageHDU(data=uncdata,name='UNCERTAINTIES',header = unchdr)

hdulist = fits.HDUList([primhdu, namehdu, spechdu, valhdu, unchdu])
hdulist.writeto('../data/galaxySEDS/'+modname+'/flux.fits',overwrite=True)


# Create models.conf file #
with open('../data/galaxySEDs/'+modname+'/models.conf','w') as modcon:
    l = ['name = '+modname+'\n', 'length_subdir = 2\n', 'aperture_dependent = no\n', 'logd_step = 0.02\n', 'version = 2\n']
    modcon.writelines(l)